In [1]:
from euroleague_api.play_by_play_data import PlayByPlay 
from euroleague_api.game_stats import GameStats
import warnings
import pandas as pd
import statsmodels.api as sm
import numpy as np
# Masquer tous les warnings
warnings.filterwarnings("ignore")
import random


C:\Users\guill\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
competition ="euroleague"
season = 2024

data_regression = pd.DataFrame(columns=["gamecode","MARKERTIME","TIME","DELTA_SCORE","PM_MINUTE"])
if competition == "euroleague" :
    pbp_ = PlayByPlay(competition="E")
    gs_ = GameStats(competition="E")

else :
    pbp_ = PlayByPlay(competition="U")
    gs_ = GameStats(competition="U")

for gc in range(9*4) : 
    gamecode=gc + 1
    print(gamecode)
    try : 
        gs = gs_.get_game_report(season=season, game_code=gamecode)
        sub = pbp_.get_game_play_by_play_data(season=season, gamecode=gamecode)

        if "End Game" in sub['PLAYINFO'].tolist() :

            HOME_TEAM = gs["local.club.code"].tolist()[0]
            AWAY_TEAM = gs["road.club.code"].tolist()[0]

            sub.loc[:, "NUMBEROFPLAY"] = range(len(sub))
            sub['PERIOD'] = sub['PERIOD'].apply(lambda x: x if x in [1, 2, 3, 4] else 4 + 0.5 * (x - 4)).astype(float)


            ##### POINTS_A

            # Remplacer les NaN au début de la colonne par 0
            first_valid_index = sub['POINTS_A'].first_valid_index()  # Trouver le premier index valide
            if first_valid_index is not None:  # S'il y a au moins un valide
                sub.loc[:first_valid_index-1, 'POINTS_A'] = sub.loc[:first_valid_index-1, 'POINTS_A'].fillna(0)

            # Remplacer les autres NaN par la dernière valeur non NaN
            sub['POINTS_A'] = sub['POINTS_A'].ffill().astype(int)


            ##### POINTS_B
            # Remplacer les NaN au début de la colonne par 0
            first_valid_index = sub['POINTS_B'].first_valid_index()  # Trouver le premier index valide
            if first_valid_index is not None:  # S'il y a au moins un valide
                sub.loc[:first_valid_index-1, 'POINTS_B'] = sub.loc[:first_valid_index-1, 'POINTS_B'].fillna(0)

            # Remplacer les autres NaN par la dernière valeur non NaN
            sub['POINTS_B'] = sub['POINTS_B'].ffill().astype(int)

            sub = sub[
                (sub['PLAYINFO']== "End Game") |
                (sub['PLAYINFO'] == 'Out') |
                (sub['PLAYINFO'] == 'In')
            ].reset_index(drop = True)

            sub['MARKERTIME'] = sub['MARKERTIME'].apply(lambda x: 0 if x == "" else int(x.split(':')[0]) + int(x.split(':')[1]) / 60)

            sub['MARKERTIME'] = 10-sub['MARKERTIME'] + 10 * (sub['PERIOD']-1)


            unique_players = sub[['PLAYER_ID', 'CODETEAM']].drop_duplicates()
            # Filtrer les événements "IN" et "OUT"
            in_events = sub[sub['PLAYTYPE'] == 'IN']
            out_events = sub[sub['PLAYTYPE'] == 'OUT']

            # Trouver le premier IN pour chaque joueur/équipe
            first_in = in_events.groupby(['PLAYER_ID', 'CODETEAM']).first().reset_index()
            first_in = first_in[['PLAYER_ID', 'CODETEAM', 'NUMBEROFPLAY']].rename(columns={'NUMBEROFPLAY': 'FIRST_IN'})

            # Trouver le premier OUT pour chaque joueur/équipe
            first_out = out_events.groupby(['PLAYER_ID', 'CODETEAM']).first().reset_index()
            first_out = first_out[['PLAYER_ID', 'CODETEAM', 'NUMBEROFPLAY']].rename(columns={'NUMBEROFPLAY': 'FIRST_OUT'})

            # Fusionner les premiers événements IN et OUT avec tous les joueurs
            result = pd.merge(unique_players, first_in, on=['PLAYER_ID', 'CODETEAM'], how='left')
            result = pd.merge(result, first_out, on=['PLAYER_ID', 'CODETEAM'], how='left')


            # Filtrer le DataFrame
            df_pbp = sub[
                (sub['PLAYINFO'] == 'Out') |
                (sub['PLAYINFO'] == 'In')
            ].reset_index(drop = True)

            df_pbp = df_pbp[["CODETEAM","PLAYER_ID","PLAYTYPE","MARKERTIME","POINTS_A","POINTS_B"]]



            result["STARTER"] = (result['FIRST_IN'] > result['FIRST_OUT']) | (result['FIRST_IN'].isna() & result['FIRST_OUT'].isna())| (result['FIRST_IN'].isna())
            starter = result[result["STARTER"]].reset_index(drop=True)
            starter = starter[starter['PLAYER_ID'] != ""]

            df_pbp_start = pd.DataFrame({
                "CODETEAM": starter["CODETEAM"].to_list(),
                "PLAYER_ID": starter["PLAYER_ID"].to_list(),
                "PLAYTYPE": ["IN"] * len(starter),
                "MARKERTIME": [0] * len(starter),
                "POINTS_A": [0] * len(starter),
                "POINTS_B": [0] * len(starter)
            })

            sub = sub[df_pbp_start.columns.tolist()]
            sub = pd.concat([df_pbp_start, sub], ignore_index=True)

            game_time = max(sub["MARKERTIME"])
            score_home = max(sub["POINTS_A"])
            score_away = max(sub["POINTS_B"])

            sub = sub[sub['PLAYTYPE']!="EG"].reset_index(drop = True)

            sub["P_T"] = sub["CODETEAM"] + "_" + sub["PLAYER_ID"]

            actual_ten = set()  # Utiliser un set pour une gestion plus rapide
            historique_ten = []

            for index, row in sub.iterrows():
                player_id = row["P_T"]
                if row["PLAYTYPE"] == "IN":
                    actual_ten.add(player_id)  # Ajouter le joueur
                elif row["PLAYTYPE"] == "OUT":
                    actual_ten.discard(player_id)  # Retirer le joueur si présent

                historique_ten.append(sorted(actual_ten))  # Append une liste triée des joueurs

            sub["TEN"] = historique_ten

            sub = sub[sub["TEN"].apply(lambda x: len(x) == 10)]

            # Garder une ligne par MARKERTIME avec l'index le plus élevé
            sub = sub.loc[sub.groupby("MARKERTIME").tail(1).index].reset_index(drop=True)[[ "MARKERTIME", "POINTS_A", "POINTS_B", "TEN"]]

            f = pd.DataFrame({
                "MARKERTIME": [game_time],
                "POINTS_A": [score_home],
                "POINTS_B": [score_away],
                "TEN": [list(actual_ten)]
            })

            sub = pd.concat([sub, f], ignore_index=True).reset_index(drop=True)

            sub['TIME'] = sub['MARKERTIME'].shift(-1) - sub['MARKERTIME']
            sub['DELTA_SCORE'] = (sub['POINTS_A'].shift(-1) - sub['POINTS_A']) - (sub['POINTS_B'].shift(-1) - sub['POINTS_B'])
            sub["PM_MINUTE"] = sub["DELTA_SCORE"] / sub["TIME"]

            sub = sub.drop(sub.index[-1]).reset_index(drop=True) 
            sub = sub.drop(["POINTS_A","POINTS_B"],axis=1)

            elements_uniques = list(set(element for sous_liste in historique_ten for element in sous_liste))


            for eu in elements_uniques :
                sub[eu] = [(int(eu in i)) * (1 if eu[:3] == HOME_TEAM else -1) for i in sub["TEN"]] 

            sub = sub.drop(["TEN"],axis=1)

            sub.insert(0,"gamecode",gamecode)
            data_regression = pd.concat([data_regression,sub])
            data_regression = data_regression.fillna(0)
    except : 
        pass


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36


In [9]:
306/35

8.742857142857142

In [4]:

# Définir les variables explicatives X (toutes sauf celles spécifiées)
X = data_regression.drop(columns=['gamecode', 'MARKERTIME', 'TIME', 'DELTA_SCORE', 'PM_MINUTE'])

# Ajout d'une constante (intercept) à X
X = sm.add_constant(X)

# Définir la variable cible Y
Y = data_regression['PM_MINUTE']

# Définir les poids
weights = data_regression['TIME']

# Ajustement du modèle de régression linéaire pondérée
model = sm.WLS(Y, X, weights=weights)
results = model.fit()


# Création du tableau des coefficients
coeffs = pd.DataFrame({
    'Variable': X.columns,
    'Coefficient': results.params,

})
# Trier le tableau des coefficients du plus grand au plus petit
coeffs_sorted = coeffs.sort_values(by='Coefficient', ascending=False)

In [7]:
coeffs_sorted.head(30)

,Coefficient,code_team,player_id
OLY_P007947,4.309701,OLY,P007947
OLY_P007982,4.258452,OLY,P007982
OLY_P001255,4.058627,OLY,P001255
RED_PJDR,3.838981,RED,PJDR
OLY_P007853,3.619836,OLY,P007853
RED_P009048,3.021096,RED,P009048
RED_P012044,3.020253,RED,P012044
PRS_P012608,2.834603,PRS,P012608
OLY_P013380,2.822723,OLY,P013380
OLY_P009849,2.735711,OLY,P009849
